# Cutting plane and bundle methods
The second part of the tutorial focuses on cutting plane and bundle
methods. We aim at resolving the following LASSO problem:
$$
\min_r \;f(r) =  \| X r + b \|_2^2 + \lambda \|r \|_1
$$
with $\lambda$ a given regularization parameter, $X$ and $b$ input data.

## Settings
We import the usual packages:

In [1]:
using Printf, Random
using LinearAlgebra
using ForwardDiff
using JuMP, CPLEX
using PyPlot

Fix seed

In [2]:
Random.seed!(2713);

Some constants

In [3]:
const LB = -1e10
const UB =  1e10
const EPS = 1e-8
const SOLVER = CPLEX.Optimizer

CPLEX.Optimizer

We first generate artificial data to study the algorithm:

In [4]:
include("data.jl")

nVariables = 10;
nCassures = 10;
xMin, xMax = build_bounds(nVariables)
A = build_X(nVariables, nCassures, xMin, xMax);
b = randn(nCassures);
λ = 50.0;

Build oracle for objective:

In [5]:
f(u) = 0.5 * norm(A*u - b, 2)^2 + λ * norm(u, 1);

## Cutting plane
The cutting plane method builds a proxy $\underline{f}_k$ for the original
function $f$, such that $\underline{f}_k$ is polyhedral and is a lower approximation:
$$
\underline{f}_k \leq f \quad \forall k
$$
If we have at disposal a collection of point $x_1, \cdots, x_k$,
with associated subgradients $g_1, \cdots, g_k$, the function
$\underline{f}_k$ writes out
$$
\begin{aligned}
f_k(x) = min_x \;& \theta  \\
         s.t. \quad & \theta \geq g_k^\top (x - x_k) + f(x_k)
\end{aligned}
$$
Using the JuMP modeler, the master problem at iteration 0 writes as

In [6]:
function init_master_cutting_plane(model, X, xMin, xMax)
    nVariables, nCassures = size(X)
    x_ = @variable(model, xMin[i] <= x_[i in 1:nVariables] <= xMax[i])
    α_ = @variable(model, α_ >= LB)
    @objective(model, Min, α_)
    return x_, α_
end;

With these two ingredients, we could define the cutting plane algorithm

In [7]:
function launch_cutting_plane(xMin, xMax, maxit=10000)
    master = Model(SOLVER)
    JuMP.set_silent(master)
    stop = false
    x, α = init_master_cutting_plane(master, xMin, xMax)
    lb, ub = LB, UB

    best_ub = ub
    
    list_ub = []
    list_lb = []
    list_best_ub = []

    for n_iter in 1:maxit
        JuMP.optimize!(master)
        lb = JuMP.value(α)
        x_k = JuMP.value.(x)
        f_k = f(x_k)
        g_k = ForwardDiff.gradient(f, x_k)
        ub = f_k
        best_ub = min(ub, best_ub)
        
        if lb >= best_ub - EPS
            break
        else
            @constraint(master, α >= f_k + dot(g_k, x - x_k))
        end
        append!(list_ub, ub)
        append!(list_lb, lb)
        append!(list_best_ub, best_ub)
    end

    return list_ub, list_lb, list_best_ub
end;

In [8]:
list_ub, list_lb, list_best_ub = launch_cutting_plane(xMin, xMax, 10000);

MethodError: MethodError: no method matching Model(::Type{CPLEX.Optimizer})
Closest candidates are:
  Model(::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at /home/lucas/.julia/packages/JuMP/MsUSY/src/JuMP.jl:126
  Model(; caching_mode, solver) at /home/lucas/.julia/packages/JuMP/MsUSY/src/JuMP.jl:163
  Model(!Matched::MathOptInterface.AbstractOptimizer, !Matched::Dict{MathOptInterface.ConstraintIndex,AbstractShape}, !Matched::Set{Any}, !Matched::Any, !Matched::Any, !Matched::Dict{Symbol,Any}, !Matched::Int64, !Matched::Dict{Symbol,Any}) at /home/lucas/.julia/packages/JuMP/MsUSY/src/JuMP.jl:126
  ...

In [9]:
list_best_ub[end] #Displays the value of min theta

UndefVarError: UndefVarError: list_best_ub not defined

In [10]:
N = length(list_best_ub)

figure()
plot((1:N), list_best_ub[1:end])
legend(["best_ub"])
title("Evolution du meilleur majorant")

figure()
semilogy((1:N), list_best_ub-list_lb))
legend(["Longueur intrvl sols"])
title("Evolution de l'amplitude l'intervalle des solutions admissibles à chaque itération")

UndefVarError: UndefVarError: list_best_ub not defined

## Bundle algorithm
Comparing to the cutting plane method, the bundle algorithm adds a
quadratic penalization to the polyhedral proxy model.
The function .
$$
\begin{aligned}
f_k(x) = min_x \;& \theta + \frac 12 \| x - x_k \|_2^2 \\
         s.t. \quad & \theta \geq g_k^\top (x - x_k) + f(x_k)
\end{aligned}
$$

In [11]:
function update_center(centre1, centre2)
    JuMP.fix.(centre1, centre2)
end


function launch_proximal(xMin, xMax, maxit=10000)
    
    nVariables = nCassures = length(xMin)
    x_k = zeros(nVariables)
    x_temp = zeros(nVariables)
    master = Model(SOLVER) 
    JuMP.set_silent(master)
    centre1 = @variable(master, centre1[1:nVariables])
    x_m = @variable(master, xMin[i] <= x_m[i in 1:nVariables] <= xMax[i])
    α = @variable(master, α >= LB)

    weight = 1.0 # weight of the regularization term
    @objective(master, Min, α + sum(weight*(centre1[i] - x_m[i])^2 for i in 1:nVariables)) 
    
    #Le problème est quadratique et ne peut pas être résolu tel quel
    
    # On passe par un deuxième problème
    model_lie = Model(SOLVER)  # cutting-plane
    JuMP.set_silent(model_lie)
    set_optimizer_attribute(model_lie, "CPX_PARAM_LPMETHOD", 2)
    g = @variable(model_lie, g >= LB)
    centre2 = @variable(model_lie, centre2[1:nVariables]) 
    @objective(model_lie, Min, g)
    stop = false
    
    lb, ub = -Inf, Inf
    # Best upper bound
    best_ub = ub
    # Number of serious step
    nb_ss = 0
    # Number of null step
    nb_ns = 0
    # Maximum number of update (i.e. center update, or serious step)
    nb_update = 3
    # Regularization weight
    weight = 1.0
    
    epsilon = 1e-5
    delta = 1e-5
    
    list_ub = []
    list_lb = []
    list_best_ub = []
    
    update_center(centre1, x_k)
    
    while !stop && nb_ns + nb_ss <= maxit && nb_ss <= nb_update

        f_temp = f(x_temp)
        grad_temp = ForwardDiff.gradient(f, x_temp)
        @constraint(master, α >= f_temp + dot(grad_temp, x_m - x_temp)) #Transformation de Yosida
        @constraint(model_lie, g >= f_temp + dot(grad_temp, centre2 - x_temp)) #Approximation tangeantielle de f
    
        update_center(centre1, x_k)

        optimize!(master)
        x_temp = JuMP.value.(x_m)
        update_center(centre2, x_temp)
        optimize!(model_lie)
        g0 = JuMP.value(g)       
        lb = g0
        ub = f(x_temp)
        best_ub = min(ub, best_ub)
        
        append!(list_ub, ub)
        append!(list_lb, lb)
        append!(list_best_ub, best_ub)

        if abs(best_ub-lb) <= weight * norm(x_k-x_temp)^2
            if norm(x_k-x_temp) <= epsilon
                stop = true
            else
                x_k = x_temp
                nb_ss += 1
            end
        else
            if abs(best_ub-lb) <= delta
                stop = true
            else
                nb_ns +=1
            end
        end
    end        
    
    return list_ub, list_lb, list_best_ub
end;



In [12]:
list_ub, list_lb, list_best_ub = launch_proximal(xMin, xMax, 10000);

MethodError: MethodError: no method matching Model(::Type{CPLEX.Optimizer})
Closest candidates are:
  Model(::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at /home/lucas/.julia/packages/JuMP/MsUSY/src/JuMP.jl:126
  Model(; caching_mode, solver) at /home/lucas/.julia/packages/JuMP/MsUSY/src/JuMP.jl:163
  Model(!Matched::MathOptInterface.AbstractOptimizer, !Matched::Dict{MathOptInterface.ConstraintIndex,AbstractShape}, !Matched::Set{Any}, !Matched::Any, !Matched::Any, !Matched::Dict{Symbol,Any}, !Matched::Int64, !Matched::Dict{Symbol,Any}) at /home/lucas/.julia/packages/JuMP/MsUSY/src/JuMP.jl:126
  ...

In [13]:
list_best_ub[end]

UndefVarError: UndefVarError: list_best_ub not defined

In [14]:
N = length(list_best_ub)

figure()
plot((1:N), list_best_ub[1:end])
legend(["best_ub"])
title("Evolution du meilleur majorant")

figure()
semilogy((1:N), list_best_ub-list_lb))
legend(["Longueur intrvl sols"])
title("Evolution de l'amplitude l'intervalle des solutions admissibles à chaque itération")

UndefVarError: UndefVarError: list_best_ub not defined